<H1>The Shortest Path Problem and Column-wise Modeling</H1>

<H3>Problem Definition</H3>

Let $N$ be a set of cities.

Let $A$ be a set of arcs between cities.

Let $d_{ij} > 0$ be the distance between city $i \in N$ and city $j \in N$.

What is the path of shortest distance between a given origin city $s \in N$ and destination city $t \in N$?

![Shortest Path Network](netflow.png)

<H3>Formulation</H3>

Let $x_{ij} = 1$ if arc $(i, j)$ is traversed.

Define the forward-star (reverse-star) of a city $i \in N$ as the set of arcs leaving (entering) city $i$:

$FS(i) = \{j | (i,j) \in A\}$

$RS(i) = \{j | (j, i) \in A\}$

\begin{eqnarray}                                                                                                                                              
\min_x && \sum_{(i,j) \in A} d_{ij} x_{ij} \nonumber \\                                                                                                       
\mbox{s.t.} && \sum_{j \in RS(i)} x_{ji} - \sum_{j \in FS(i)} x_{ij} = b_i,\;\;i \in N \nonumber \\                                                           
&& 0 \le x_{ij} \le 1,\;\;(i,j) \in A, \nonumber                                                                                                              
\end{eqnarray}                                                                                                                                                
where $b_s = -1$, $b_t = 1$, and $b_i = 0$ for $i \neq s, t$.

If we travel along arc $(i, j)$, we add a distance $d_{ij}$ to our journey. The objective is to minimize the total distance travelled. The constraints enforce a balance of flow at each city. The sum $\sum_{j \in RS(i)} x_{ji}$ counts the number of times we enter city $i \in N$, and the sum $\sum_{j \in FS(i)} x_{ij}$ counts the number of times we leave city $i \in N$. The difference between those two sums should be -1 for the source city $s \in N$ (we leave $s$ once and never return), 1 for the destination city $t \in N$ (we enter it once but never leave), and 0 for all cities in between (either we never visit the city, or we enter and leave the city exactly once).

<H3>Inputs</H3>

We'll need a list of the arcs in our network, along with the distance associated with each arc. Here, an arc can be represented as a pair of cities.

In [ ]:
data = """Honolulu, Chicago, 105
Honolulu, San Francisco, 75
Honolulu, Los Angeles, 68 
Chicago, Boston, 45 
Chicago, New York, 56 
San Francisco, Boston, 71 
San Francisco, New York, 48 
San Francisco, Atlanta, 63 
Los Angeles, New York, 44 
Los Angeles, Atlanta, 57 
Boston, Heathrow London, 88 
New York, Heathrow London, 65 
Atlanta, Heathrow London, 76
"""

We will use the pandas library to help read the data.

In [ ]:
from StringIO import StringIO
import pandas as pd

In [ ]:
arcs = pd.read_csv(StringIO(data), header=None, skipinitialspace=True)
arcs.columns = ['source', 'destination', 'distance']
print arcs

In [ ]:
cities = pd.concat([arcs.source, arcs.destination]).unique()
print cities
arcs = [row for i, row in arcs.iterrows()]

<H3>Building the Model: Naive Approach</H3>

The flow balance constraints in the above shortest path formulation involve sums over dynamic sets $RS(i)$ and $FS(i)$, which are both subsets of the full arc set $A$, and also vary depending on the city $i$. This is a departure from, say, the diet problem in which every constraint has a sum over the full set of food types. As such, we'll need to take a slightly different approach in order to efficiently add constraints to the model. We'll start off by presenting a naive approach that is $O(|N|*|A|)$ just to build the model.

We'll first add variables and set the objective.

In [ ]:
import gurobipy as grb

m = grb.Model()
arc_traversed = {(arc.source, arc.destination): m.addVar(obj=arc.distance,
                                                        name='.'.join(('arc_traversed', arc.source, arc.destination)))
                 for arc in arcs}
m.update()
print arc_traversed

We now have a dictionary that maps a pair of cities to the decision variable which will ultimately tell use if we travel between those cities. Each city $i \in N$ will get its own flow balance constraint. To write that constraint we need to know what arcs enter and leave the city, and that requires a lookup of the arcs set. We can write methods to do this for us.

In [ ]:
def get_forward_star(arc_traversed, city):
    return [var for (source, destination), var in arc_traversed.iteritems() if source == city]

def get_reverse_star(arc_traversed, city):
    return [var for (source, destination), var in arc_traversed.iteritems() if destination == city]

Let's pick an intermediate city, say Atlanta, and generate the linear expression for its flow balance constraint.

In [ ]:
num_times_leaving = get_forward_star(arc_traversed, 'Atlanta')
num_times_entering = get_reverse_star(arc_traversed, 'Atlanta')
balance_expr = grb.quicksum(num_times_entering) - grb.quicksum(num_times_leaving)
print balance_expr

This looks correct. The forward star consists of the single arc to Heathrow London, and the reverse star consists of the arcs from Los Angeles and San Francisco. We can only travel from Atlanta to Heathrow London if we first travelled to Atlanta via either Los Angeles or San Francisco, so this expression needs to be equal to 0. A naive approach will simply repeat this lookup for every city.

In [ ]:
def naive_shortest_path(cities, arcs, origin, destination):
    def get_forward_star(city):
        return [var for (source, destination), var in arc_traversed.iteritems() if source == city]
    
    def get_reverse_star(city):
        return [var for (source, destination), var in arc_traversed.iteritems() if destination == city]    
    
    model = grb.Model()

    arc_traversed = {(arc.source, arc.destination): model.addVar(obj=arc.distance,
                                                        name='.'.join(('arc_traversed', arc.source, arc.destination)))
                     for arc in arcs}
    model.update()
    flow_balance = {}
    for city in cities:
        num_times_leaving = grb.quicksum(get_forward_star(city))
        num_times_entering = grb.quicksum(get_reverse_star(city))
        flow_balance[city] = model.addConstr(num_times_entering - num_times_leaving == 0,
                                             name='flow.' + str(city))
    model.update()
    flow_balance[origin].RHS = -1
    flow_balance[destination].RHS = 1
    model.optimize()
    model.write('naive.lp')
    for arc, var in arc_traversed.iteritems():
        if var.X > 0.5:
            print arc
    print "Total distance =", model.ObjVal

In [ ]:
naive_shortest_path(cities, arcs, 'Honolulu', 'Heathrow London')

In [ ]:
!cat naive.lp

We can check the lp file to verify that this approach did actually build the correct model. However, we are doing two lookups on the arc list every time we add a flow balance constraint for a city, which is needlessly inefficient. Let's look at some ways of eliminating this inefficiency.

<H3>Modeling with tuplelist</H3>

Many optimization models involve sums over dynamic sets such as forward star and reverse star above. When each constraint uses a different subset of variables, we should take care to ensure that we aren't doing an inefficient lookup of that subset.

It is commonplace to have a group of variables with a pair or tuple for an index, and constraints that fix some elements of the tuple and sum over the remaining elements. For the shortest path problem, each variable is indexed by two cities, the source and the destination, and the sums in the constraints either fix the source and sum over the destinations, or vice versa. 

It's important to efficiently determine which variables participate in each constraint. Fortunately, this pattern shows up frequently enough in optimization models that Gurobi has provided a special data structure called a tuplelist which does exactly what we need quickly and with clean syntax that reads close to a pure modeling language.

The list of tuples that you create a tuplelist out of will typically be the key set of a decision variable dictionary. In the shortest path example, each key in our arc_traversed dictionary is a source, destination city pair.

In [ ]:
arc_tuple_list = grb.tuplelist([(arc.source, arc.destination) for arc in arcs])
print arc_tuple_list

Since every element of arc_tuple_list is a key in the arc_traversed dictionary, if we select a subset of these tuples, we can use them to sum over a subset of the Var objects in the dictionary. Every tuplelist has a select method which selects the subset of tuples which matches a pattern. The pattern to match on can include wildcards for any element of the tuple. For example, here's how we'd use the select method to pull the set of inbound and outbound arcs for Atlanta:

In [ ]:
print arc_tuple_list.select('*', 'Atlanta')
print arc_tuple_list.select('Atlanta', '*')

The tuplelist data structure exists to do this particular type of lookup, so the select method is much faster than the brute force lookups we performed in the naive implementation. Let's replace those lookups with tuplelist's select method.

In [ ]:
def tuplelist_shortest_path(cities, arcs, origin, destination):   
    model = grb.Model()

    arc_traversed = {(arc.source, arc.destination): model.addVar(obj=arc.distance,
                                                        name='.'.join(('arc_traversed', arc.source, arc.destination)))
                     for arc in arcs}
    model.update()
    
    arc_tuple_list = grb.tuplelist(arc_traversed.keys())    
    flow_balance = {}
    for city in cities:
        num_times_leaving = grb.quicksum(arc_traversed[arc] for arc in arc_tuple_list.select(city, '*'))
        num_times_entering = grb.quicksum(arc_traversed[arc] for arc in arc_tuple_list.select('*', city))
        flow_balance[city] = model.addConstr(num_times_entering - num_times_leaving == 0,
                                             name='flow.' + str(city))
    model.update()
    flow_balance[origin].RHS = -1
    flow_balance[destination].RHS = 1
    model.optimize()
    model.write('tuplelist.lp')
    for arc, var in arc_traversed.iteritems():
        if var.X > 0.5:
            print arc
    print "Total distance =", model.ObjVal

In [ ]:
tuplelist_shortest_path(cities, arcs, 'Honolulu', 'Heathrow London')

In [ ]:
!cat tuplelist.lp

<H3>Modeling with Columns</H3>

When building a Gurobi model, it can be useful to focus on a single constraint or row, and ask what variables participate in that constraint. 

While this row-wise view is frequently the most intuitive view of the model, there are some cases in which it is difficult to describe the set of variables corresponding to a constraint. In these cases it may actually be easier to focus on a single variable or column, and ask what constraints that variable participates in. That is, we may wish to think about what a specific column in the constraint matrix looks like.

Both the row-wise and column-wise approaches can lead us to a correct representation of the model, there are reasons to prefer one over the other depending on the context. Generally speaking, we will prefer the column-wise perspective when each variable appears in a fixed number of constraints.

In the context of the shortest path formulation above, each variable indicates whether we traverse a particular arc, and each constraint enforces a conservation of flow for a particular city. Consider one of these constraints, say the one corresponding to Chicago. Each of the rows in the arc DataFrame above corresponds to a decision variable. We can certainly look up the arcs that are incident to Chicago -- there is an inbound arc from Honolulu and two outbound arcs to Boston and New York -- and write down a constraint that says that we only traverse an outbound arc if we traversed an inbound arc. We could repeat this arc lookup for each city to build the full model, with each lookup returning a different subset of arcs depending on the city.

Now consider one of the variables in the model, say the one corresponding to the arc from Los Angeles to New York. Determining which constraints this variable participates in requires no lookup. It participates in the constraint for Los Angeles and the constraint for New York. In fact, every variable in the model participates in exactly two constraints-- one for the source and one for the destination city. If we follow the convention that flow in to a city is positive and flow out is negative, each column in the constraint matrix has exactly two non-zero entries, a 1 for the destination city and a -1 for the source city. That each column has a very easily-defined structure is an indication that the model is amenable to a column-wise approach.

Let's create a column for our shortest path model. Gurobi provides a Column object for this purpose, which takes in a list of coefficients and a list of constraints. This means we'll need to add the constraints to the model before we create columns. When we create the constraints, we intentionally omit the variables as they haven't even been created yet. This can be counterintuitive but will make sense once we demonstrate it.

In [ ]:
m = grb.Model()
flow_balance = {city: m.addConstr(grb.LinExpr() == 0, name='flow.' + str(city)) for city in cities}
m.update()
print flow_balance

Now let's create the column for the Los Angeles to New York arc traversal variable. This variable contributes a +1 to the New York flow constraint, and a -1 to the Los Angeles flow constraint.

In [ ]:
column = grb.Column(coeffs=[1, -1], constrs=[flow_balance['New York'], flow_balance['Los Angeles']])
print column

The Column object looks a lot like the LinExpr object, except that it represents a list of constraints instead of variables. And, where we normally use a LinExpr when we call addConstr, we can pass a Column in as a parameter to addVar.

In [ ]:
var = m.addVar(obj=44, name='arc_traversed.Los_Angeles.New_York', column=column)

This will retroactively add the variable we just created to the two constraints it participated in. If we do this in a loop over all arcs in the network, we can build the full shortest path model.

In [ ]:
def solve_shortest_path(cities, arcs, origin, destination):
    def get_column(arc):
        return grb.Column(coeffs=[1, -1], constrs=[flow_balance[arc.destination], flow_balance[arc.source]])

    model = grb.Model()
    flow_balance = {city: model.addConstr(grb.LinExpr() == 0, name='flow.' + str(city))
                    for city in cities}
    model.update()
    flow_balance[origin].RHS = -1
    flow_balance[destination].RHS = 1
    
    arc_traversed = {(arc.source, arc.destination): model.addVar(obj=arc.distance,
                                                                 name='.'.join(('arc_traversed', arc.source, arc.destination)),
                                                                 column=get_column(arc))
                     for arc in arcs}
    model.optimize()
    for arc, var in arc_traversed.iteritems():
        if var.X > 0.5:
            print arc
    print "Total distance =", model.ObjVal

In [ ]:
solve_shortest_path(cities, arcs, 'Honolulu', 'Heathrow London')